In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "question": "Can you sum up the movie Top Gun in three emojis?",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "question": "Can you sum up the movie The Godfather in three emojis?",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
    {
        "question": "Can you sum up the movie Fight Club in three emojis?",
        "answer": """
        👊💥🤼
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}")
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain("The Matrix")

content='🕶️💊🤖'


In [2]:
invoke_chain("Inception")

content='🕰️💭🔁'


In [3]:
invoke_chain("What was that movie I just mentioned first?")

content='You mentioned "The Matrix" first.'
